# Thesis: Code Samples

This notebook is for my ([Alex Wendland](https://blog.alexwendland.com/)) undergraduate thesis for Honors in Computer Science at Harvard called [WebAssembly as a Multi-Language Platform](https://github.com/awendland/2020-thesis).

The following code samples are taken from the thesis and made runnable inside this Jupyter notebook via [wasm-spec-kernel](https://github.com/awendland/wasm_spec_kernel).

For my thesis I added support for abstract types (similar to [OCaml's abstract types](https://ocaml.org/learn/tutorials/modules.html#Abstract-types)) to the [reference interpreter](https://github.com/WebAssembly/spec/tree/master/interpreter) for WebAssembly. My extended interpreter can be found at [awendland/webassembly-spec-abstypes](https://github.com/awendland/webassembly-spec-abstypes).
Each demo will specify if they need this language extension by saying `leverages Core WebAssembly + abstract types`. Demos that don't include `+ abstract types` can be run using an WebAssembly v1 compliant engine.
If you launched this notebook via the Binder link in the README, wasm-spec-kernel will already be properly configured to use my extended interpreter. Otherwise, see the [wasm-spec-kernel repo](https://github.com/awendland/wasm_spec_kernel) for configuration instructions.


### Demo 2.2 - example date library

NOTE: This sample is a poor implementation for a date library because it incorrectly assumes that 1 year is always 31,557,600,000 milliseconds. Do not use it.

This is related to Section 3.2 in the thesis (and also Section 3.4). It leverages Core WebAssembly + abstract types (https://github.com/awendland/webassembly-spec-abstypes).

In [ ]:
(module $lib_date
  (abstype_new $Date i32)
  (func (export "createDate")
    (param $day i32) (param $month i32) (param $year i32)
    (result (abstype_new_ref $Date))
    (i32.add ;; Day, Mon, Year -> Unix milliseconds
      (i32.mul (local.get $day) (i32.const 86400))
      (i32.add
        (i32.mul (local.get $month) (i32.const 2592000))
        (i32.mul (i32.const 31557600)
          (i32.sub (local.get $year) (i32.const 1970)))))
  )
  (func (export "yearsBetweenDates") (param (abstype_new_ref $Date))
    (param (abstype_new_ref $Date)) (result i32)
    (i32.sub (local.get 0) (local.get 1))
    (i32.div_s (i32.const 31557600))
  )
  (export "Date" (abstype_new_ref $Date))
)
(register "lib_date" $lib_date)

(module $main
  (import "lib_date" "Date" (abstype_sealed $Date))
  (import "lib_date" "createDate" (func $createDate
    (param i32) (param i32) (param i32)
    (result (abstype_sealed_ref $Date))))
  (import "lib_date" "yearsBetweenDates" (func $yearsBetweenDates
    (param (abstype_sealed_ref $Date))
    (param (abstype_sealed_ref $Date)) (result i32)))
  (func (export "main") (result i32)
    (call $createDate
      (i32.const 2) (i32.const 20) (i32.const 1962))
    (call $createDate
      (i32.const 8) (i32.const 26) (i32.const 1918))
    (call $yearsBetweenDates)
  )
)
(assert_return (invoke $main "main") (i32.const 43))

#### Comparable OCaml

```ocaml
(* lib.ml *)
module Date = sig
  type date (* public, abstract type *)
end =
struct
  type date = {day : int;  month : int;  year : int} (* private, concrete type *)
  val create : ?days:int -> ?months:int -> ?years:int -> unit -> date
  val yearsBetweenDates : date -> date -> int
  val month : date -> int
  ...
end

(* consumer.ml *)
let kjohnson_bday : Date.date = Date.create 8 26 1918 () in
let mercury_launch : Date.date = Date.create 2 20 1962 () in
let kj_age_at_launch = Date.yearsBetweenDates kjohnson_bday mercury_launch in ...
(* kjognson_bday.day <- this access is invalid *)
```

#### Alternative Abstract Type Syntax

```commonlisp
(module $lib_date
  (export "Date" (newtype $Date i32))
  (func (export "createDate")
    (param $day i32) (param $month i32) (param $year i32) (result (type $Date))
    (i32.add
      (i32.mul (local.get $day) (i32.const 86400))
      (i32.add
        (i32.mul (local.get $month) (i32.const 2592000))
        (i32.mul (i32.const 31557600)
          (i32.sub (local.get $year) (i32.const 1970))
    )))
  )
  (func (export "yearsBetweenDates")
    (param (type $Date)) (param (type $Date)) (result i32)
    (i32.sub (local.get 0) (local.get 1))
    (i32.div_s (i32.const 31557600))
  )
)
(register "lib_date" $lib_date)

(module $main
  (import "lib_date" "Date" (type $Date))
  (import "lib_date" "createDate" (func $createDate
    (param i32) (param i32) (param i32) (result (type $Date))))
  (import "lib_date" "yearsBetweenDates" (func $yearsBetweenDates
    (param (type $Date)) (param (type $Date)) (result i32)))
  (func (export "main") (result i32)
    (call $createDate
      (i32.const 2) (i32.const 20) (i32.const 1962))
    (call $createDate
      (i32.const 8) (i32.const 26) (i32.const 1918))
    (call $yearsBetweenDates)
  )
)
```

### Demo 3.1 - call foreign function



This is related to Section 3.1 in the thesis. It leverages Core WebAssembly.

In [ ]:
;; /* lib.cpp */
;; bool isEven(int a) {
;;   return a % 2 == 0;
;; }

(module $demo01_m1
  (func $isEven (export "isEven") (param i32)
    (result i32) ;; i32 is bool (0=false, 1=true)
    (i32.rem_u (local.get 0) (i32.const 2))
    (i32.const 0)
    (i32.eq))
)
(register "demo01_m1" $demo01_m1)

;; /* main.rs */
;; 
;; extern "WASM" {
;;   pub fn isEven(a: i32) -> bool;
;; }
;;
;; pub fn main() -> bool {
;;   return isEven(4) == true; // assert
;; }

(module $demo01_m2
  (type (;0;) (func (param i32) (result i32)))
  (import "demo01_m1" "isEven" (func $isEven (type 0)))
  (func $main (export "main") (result i32)
    (i32.const 4)
    (call $isEven)
    (i32.eq (i32.const 1 (;true;))))
)
(register "demo01_m2" $demo01_m2)

(assert_return (invoke $demo01_m2 "main") (i32.const 1 (;true;)))

### Demo 3.2 - pass foreign function to other foreign function



This is related to Section 3.2 in the thesis. It leverages Core WebAssembly.

In [ ]:
;; /* lib.cpp */
;; bool isEven(int a) {
;;   return a % 2 == 0;
;; }

(module $demo02_m1
  (func $isEven (export "isEven") (param i32)
    (result i32) ;; i32 is bool (0=false, 1=true)
    (i32.rem_u (local.get 0) (i32.const 2))
    (i32.const 0)
    (i32.eq))
)
(register "demo02_m1" $demo02_m1)

;; /* lib.zig */
;; const const num: i32 = 53
;;
;; export fn test_num(pred: fn(i32) -> bool) -> bool {
;;   return pred(elem);
;; }

(module $demo02_m2
  (global $num i32 (i32.const 53))
  (type (;0;) (func (param i32) (result i32)))
  ;; create a table which the predicate function will be provided through
  (table $fns (export "_fns") 1 funcref)
  ;; get the index of the slot to register a func in
  (func (export "_fns_slot") (result i32) (i32.const 0))
  ;; free up the slot
  (func (export "_fns_free") (param $slot i32)
    (table.set $fns (local.get $slot) (ref.null)))
  (func $test_num (export "test_num") (param $_fn_slot i32) (result i32)
    (global.get $num)
    ;; call the predicate fn
    (call_indirect $fns (type 0) (local.get $_fn_slot)))
)
(register "demo02_m2" $demo02_m2)

;; /* main.rs */
;; 
;; extern "WASM" {
;;   pub fn test_num(pred: &dyn Fn(i32) -> bool) -> bool;
;;   pub fn isEven(a: i32) -> bool;
;; }
;;
;; pub fn main() -> bool {
;;   return test_num(isEven);
;; }

(module $demo02_m3
  (type (func (param i32) (result i32))) ;; 0
  (import "demo02_m1" "isEven" (func $isEven (type 0)))
  (import "demo02_m2" "test_num" (func $test_num (type 0)))
  (import "demo02_m2" "_fns" (table $m2_fns 1 funcref))
  (import "demo02_m2" "_fns_slot" (func $m2_fns_slot (result i32)))
  (import "demo02_m2" "_fns_free" (func $m2_fns_free (param i32)))
  ;; register the func as exportable
  (elem declare func $isEven $isEven)
  (func $main (export "main") (result i32) (local i32 i32)
    ;; pass the predicate func to $demo_m2
    (local.set 0 (call $m2_fns_slot))
    (table.set $m2_fns (local.get 0) (ref.func $isEven))
    ;; call the func
    (local.set 1 (call $test_num (local.get 0)))
    ;; cleanup the predicate func
    (call $m2_fns_free (local.get 0))
    (local.get 1))
)
(register "demo02_m3" $demo02_m3)

(assert_return (invoke $demo02_m3 "main") (i32.const 0 (;false;)))

### Demo 3.6 - simplified fraction representation invariant



This is related to Section 3.6 in the thesis. It leverages Core WebAssembly + abstract types (https://github.com/awendland/webassembly-spec-abstypes).

In [ ]:
;; Simple gcd implementation to polyfill C++ std::gcd
(module $demo03_std
  (func $gcd (export "gcd") (param $a i32) (param $b i32) (result i32)
    (if (result i32) (i32.eq (local.get $a) (i32.const 0))
      (then (local.get $b))
      (else
        (call $gcd
            (i32.rem_s (local.get $b) (local.get $a))
            (local.get $a)))))
)
(register "std" $demo03_std)

;; /* demo03_m1.cpp */
;; class RationalNum {
;;   private:
;;     int _num, _den;
;;   public:
;;     RationalNum(int num, int den) {
;;       int gcd = std::gcd(num, den);
;;       _num = num / gcd;
;;       _den = den / gcd;
;;     }
;; }

(module $demo03_m1
  (import "std" "gcd" (func $_std_gcd (param i32) (param i32) (result i32)))
  (memory 1)
  (func $_malloc (param i32) (result i32)
    ;; Only support a single allocation for this toy example
    ;; i.e. ignore size and return the same i32 address every time
    (i32.const 0))
  (abstype_new $RationalNum i32)
  ;; RationalNum struct = {int, int} = 4 + 4 = 8 bytes
  (func (export "RationalNum.new") (param $num i32) (param $den i32)
    (result (abstype_new_ref $RationalNum)) (local $gcd i32) (local $adr i32)
    (local.set $gcd (call $_std_gcd (local.get $num) (local.get $den)))
    (local.set $adr (call $_malloc (i32.const 8)))
    (i32.store offset=0 (local.get $adr)
      (i32.div_s (local.get $num) (local.get $gcd)))
    (i32.store offset=4 (local.get $adr)
      (i32.div_s (local.get $num) (local.get $gcd)))
    (local.get $adr))
  (func (export "RationalNum.getNumerator")
    (param $this (abstype_new_ref $RationalNum)) (result i32)
    (i32.load offset=0 (local.get $this)))
  (export "RationalNum" (abstype_new_ref $RationalNum))
)
(register "demo03_m1" $demo03_m1)


(module $demo03_test
  (import "demo03_m1" "RationalNum" (abstype_sealed $RationalNum))
  (import "demo03_m1" "RationalNum.new" (func $RationalNum.new
    (param i32) (param i32) (result (abstype_sealed_ref $RationalNum))))
  (import "demo03_m1" "RationalNum.getNumerator" (func $RationalNum.getNumerator
    (param (abstype_sealed_ref $RationalNum)) (result i32)))
  (func (export "main") (result i32)
    (local $ratio (abstype_sealed_ref $RationalNum))
    (local.set $ratio
      (call $RationalNum.new (i32.const 10) (i32.const 2)))
    (; e.g. perform other operations w/ $ratio (which is "5/2") ;)
    (call $RationalNum.getNumerator (local.get $ratio))
  )
)

(assert_return (invoke $demo03_test "main") (i32.const 5))

### Demo 4.2 - array access



This is related to Section 3.6 and Appendix A.2 in the thesis. It leverages Core WebAssembly + abstract types (https://github.com/awendland/webassembly-spec-abstypes).

In [ ]:
(module $lib_buffer
  (memory 1)
  (global $nextAddr (mut i32) (i32.const 0))
  (abstype_new $Buffer i32) ;; a sequence of bytes
  (func $Buffer.create (param $size i32) (result (abstype_new_ref $Buffer))
    (local i32)
    (local.set 1 (global.get $nextAddr))
    (i32.store (local.get 1) (local.get $size))
    (global.set $nextAddr
      (i32.add (local.get 1)
        (i32.add (local.get $size) (i32.const 4))))
    (local.get 1))
  (func $Buffer.size (param $this (abstype_new_ref $Buffer)) (result i32)
    (i32.load (local.get 0)))
  (func $Buffer.i32_load (param $this (abstype_new_ref $Buffer)) (param $idx i32) (result i32)
    (i32.add (i32.add (local.get $this) (i32.const 4)) (local.get $idx))
    (i32.load))
  (func $Buffer.i32_load8_u (param $this (abstype_new_ref $Buffer)) (param $idx i32) (result i32)
    (i32.add (i32.add (local.get $this) (i32.const 4)) (local.get $idx))
    (i32.load8_u))
  (func $Buffer.i32_store (param $this (abstype_new_ref $Buffer)) (param $idx i32) (param $data i32)
    (i32.add (i32.add (local.get $this) (i32.const 4)) (local.get $idx))
    (local.get $data)
    (i32.store))
  (func $Buffer.i32_store8 (param $this (abstype_new_ref $Buffer)) (param $idx i32) (param $data i32)
    (i32.add (i32.add (local.get $this) (i32.const 4)) (local.get $idx))
    (local.get $data)
    (i32.store))
  (abstype_new $ReadonlyBuffer i32)
  (func $ReadonlyBuffer.fromBuffer (param $super (abstype_new_ref $Buffer))
    (result (abstype_new_ref $ReadonlyBuffer))
    (local.get 0))
  (func $ReadonlyBuffer.i32_load (param $this (abstype_new_ref $Buffer)) (param $idx i32) (result i32)
    (call $Buffer.i32_load (local.get $this) (local.get $idx)))
  (func $ReadonlyBuffer.i32_load8_u (param $this (abstype_new_ref $Buffer)) (param $idx i32) (result i32)
    (call $Buffer.i32_load8_u (local.get $this) (local.get $idx)))
)
(register "lib_buffer" $lib_buffer)